In [1]:
import os
os.chdir("../")
%pwd

'e:\\Deep Learning\\TENSORFLOW\\rice_image_detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelPreparationConfig:
    root_dir: Path
    model_dir: Path
    weight_decay: float
    input_image_size: list
    learning_rate: float
    epsilon: float
    classes: int
    epochs: int
    decay_rate: float

In [6]:
from src.RICE_IMAGE_DETECTION.constants import *
from src.RICE_IMAGE_DETECTION.utils.common import read_yaml, create_directories

In [8]:
class ConfigureationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_preparation_config(self) -> ModelPreparationConfig:
        config = self.config.prepare_model

        create_directories([config.root_dir])

        model_preparation_config = ModelPreparationConfig(
            root_dir = Path(config.root_dir),
            model_dir = Path(config.model_dir),
            weight_decay = self.params.weight_decay,
            input_image_size= self.params.input_image_size,
            learning_rate = self.params.learning_rate,
            epsilon = self.params.epsilon,
            classes = self.params.classes,
            epochs = self.params.epochs,
            decay_rate = self.params.decay_rate
        )

        return model_preparation_config

In [9]:
import os
import tensorflow as tf
from tensorflow import keras
from src.RICE_IMAGE_DETECTION import logger

In [10]:
class ModelPreparation:
    def __init__(self, config: ModelPreparationConfig):
        self.config = config

    def model(self):
        logger.info(f"------------Model Preparation Started-------------")
        classes = self.config.classes
        weight_decay = self.config.weight_decay
        input_shape = self.config.input_image_size
        cnn = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape,
                                kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
            tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                                kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
            tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=128, activation='relu',
                                kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
            tf.keras.layers.Dense(classes, activation='softmax')
            ])
        logger.info(f"------------Model Preparation Ended-------------")
        return cnn
    
    def model_compilation(self, model):
        decay_rate = self.config.decay_rate
        epsilon = self.config.epsilon
        learning_rate = self.config.learning_rate
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=learning_rate, decay_steps=100000, decay_rate=decay_rate, staircase=True)
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, epsilon=epsilon)
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model
    
    def save_model(self, model):
        model_dir = Path(self.config.model_dir)
        model.save(model_dir)
        logger.info(f"---MODEL SAVED------")


In [11]:
try:
    config = ConfigureationManager()
    model_preparation_config = config.get_model_preparation_config()
    model_preparation = ModelPreparation(config=model_preparation_config)
    cnn = model_preparation.model()
    model = model_preparation.model_compilation(cnn)
    model_preparation.save_model(model)
    logger.info(f"---MODEL SUMMARY------")
    logger.info(cnn.summary())
except Exception as e:
    raise e

[2024-07-01 10:22:13,094: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-01 10:22:13,097: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-01 10:22:13,097: INFO: common: created directory at: artifacts]
[2024-07-01 10:22:13,097: INFO: common: created directory at: artifacts/prepare_model]
[2024-07-01 10:22:13,097: INFO: 3139924240: ------------Model Preparation Started-------------]
[2024-07-01 10:22:13,397: INFO: 3139924240: ------------Model Preparation Ended-------------]
[2024-07-01 10:22:15,314: INFO: 3139924240: ---MODEL SAVED------]
[2024-07-01 10:22:15,315: INFO: 1108546764: ---MODEL SUMMARY------]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0 